### FIRMS Wildfire Detections and Data Application

Vamos a realizar un script que mediante una API de la NASA nos devuelve un dataframe con los datos de fuego detectados por satélite en una zona y los mostraremos en un mapa.

Comenzamos importando las librerias y realizando unas consultas a la API para saber las transacciones que hemos realizado.

In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

MAP_KEY = 'a0f1afd59cc42db75820a429a7617f3a'

# Para comprobar las transacciones de la API
url = 'https://firms.modaps.eosdis.nasa.gov/mapserver/mapkey_status/?MAP_KEY=' + MAP_KEY
try:
  df = pd.read_json(url,  typ='series')
  display(df)
except:
  print ("There is an issue with the query. \nTry in your browser: %s" % url)

# La función para comprobar las transacciones que hemos realizado
def get_transaction_count() :
  count = 0
  try:
    df = pd.read_json(url,  typ='series')
    count = df['current_transactions']
  except:
    print ("Error in our call.")
  return count

tcount = get_transaction_count()
print ('Our current transaction count is %i' % tcount)

transaction_limit             5000
current_transactions           185
transaction_interval    10 minutes
dtype: object

Our current transaction count is 185


Ahora realizamos una consulta para saber las tecnologías dispinibles para luego poder elegir una para realizar la consulta.

In [3]:
# URL para consultar la disponibilidad de los sensores y su disponibilidad.
sensors_url = f'https://firms.modaps.eosdis.nasa.gov/api/data_availability/csv/{MAP_KEY}/all'

# Leer el CSV desde la URL.
sensors_df = pd.read_csv(sensors_url)

# Mostrar los datos disponibles.
display(sensors_df)

,data_id,min_date,max_date
0,MODIS_NRT,2024-08-01,2024-12-01
1,MODIS_SP,2000-11-01,2024-07-31
2,VIIRS_NOAA20_NRT,2019-12-04,2024-12-01
3,VIIRS_NOAA21_NRT,2024-01-17,2024-12-01
4,VIIRS_SNPP_NRT,2024-07-01,2024-12-01
5,VIIRS_SNPP_SP,2012-01-20,2024-06-30
6,LANDSAT_NRT,2022-06-20,2024-12-01
7,GOES_NRT,2022-08-09,2024-12-01
8,BA_MODIS,2000-11-01,2024-07-01


Podemos elegir una tecnología, en este caso, VIIRS_NOAA21_NRT, que aporta datos NRT (Near Real Time) Near real-time (NRT) Suomi National Polar-orbiting Partnership (Suomi NPP) Visible Infrared Imaging Radiometer Suite (VIIRS) que a diferencia de otras tecnologías ofrece una banda de 375m que permite localizar fuegos más pequeños.

Para ello, debemos establecer la estructura de la query que es API + MAP_KEY + MODEL + ZONA + Nº de dias desde la fecha de solicitud + Fecha (Opcional)

In [4]:
MAP_KEY = 'a0f1afd59cc42db75820a429a7617f3a'
model = "VIIRS_NOAA21_NRT"
zone = "world"
days_num = 1

#Tener en cuenta que en esta api se está poniendo "area"
area_url = f'https://firms.modaps.eosdis.nasa.gov/api/area/csv/{MAP_KEY}/{model}/{zone}/{days_num}'
start_count = get_transaction_count()
df_area = pd.read_csv(area_url)
end_count = get_transaction_count()
print ('We used %i transactions.' % (end_count-start_count))

df_area

We used 36 transactions.


,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight
0,58.89273,52.47691,297.04,0.59,0.70,2024-12-01,39,N21,VIIRS,n,2.0NRT,265.98,1.58,N
1,59.05310,52.32759,302.90,0.57,0.69,2024-12-01,39,N21,VIIRS,n,2.0NRT,264.98,1.56,N
2,48.60738,21.18103,321.49,0.38,0.36,2024-12-01,41,N21,VIIRS,n,2.0NRT,275.92,1.68,N
3,48.61953,21.19090,295.10,0.38,0.36,2024-12-01,41,N21,VIIRS,n,2.0NRT,272.16,0.68,N
4,49.21681,16.77592,297.16,0.44,0.39,2024-12-01,41,N21,VIIRS,n,2.0NRT,272.83,0.62,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27868,-3.35200,-39.14245,333.59,0.55,0.68,2024-12-01,1520,N21,VIIRS,n,2.0NRT,296.48,5.90,D
27869,-3.32866,-40.09351,367.00,0.64,0.72,2024-12-01,1520,N21,VIIRS,h,2.0NRT,295.62,19.49,D
27870,-3.24847,-40.80051,354.74,0.71,0.75,2024-12-01,1520,N21,VIIRS,n,2.0NRT,289.85,34.75,D
27871,-3.17759,-40.94885,332.13,0.73,0.76,2024-12-01,1520,N21,VIIRS,n,2.0NRT,291.78,9.14,D


Podemos sacar un df con datos de localizaciones de paises

In [5]:
countries_url = 'https://firms.modaps.eosdis.nasa.gov/api/countries'
df_countries = pd.read_csv(countries_url, sep=';')
df_countries

,id,abreviation,name,extent
0,1,ABW,Aruba,"BOX(-70.0624080069999 12.417669989,-69.8768204..."
1,2,AFG,Afghanistan,"BOX(60.4867777910001 29.3866053260001,74.89230..."
2,3,AGO,Angola,"BOX(11.6693941430001 -18.0314047239998,24.0617..."
3,4,AIA,Anguilla,"BOX(-63.4288223949999 18.1690941430001,-62.972..."
4,6,ALA,Aland Islands,"BOX(19.5131942070001 59.9044863950001,21.09669..."
...,...,...,...,...
239,234,WSM,Samoa,"BOX(-172.782582161 -14.052829685,-171.43769283..."
240,235,YEM,Yemen,"BOX(42.5457462900001 12.1114436720001,54.54029..."
241,236,ZAF,South Africa,"BOX(16.4699813160001 -46.965752863,37.97779381..."
242,237,ZMB,Zambia,"BOX(21.9798775630001 -18.0692318719999,33.6742..."


Para sacar el df de un pais concreoto, usamos la API de countries y le pasamos el codigo de país

In [6]:
MAP_KEY = 'a0f1afd59cc42db75820a429a7617f3a'
model = "VIIRS_NOAA21_NRT"
zone = "ESP"
days_num = 1

spain_url = f'https://firms.modaps.eosdis.nasa.gov/api/country/csv/{MAP_KEY}/{model}/{zone}/{days_num}'
df_spain = pd.read_csv(spain_url)

df_spain

,country_id,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight
0,ESP,37.35946,-5.86398,298.04,0.43,0.38,2024-12-01,228,N21,VIIRS,n,2.0NRT,282.51,1,N
1,ESP,37.40304,-5.75517,297.74,0.43,0.38,2024-12-01,228,N21,VIIRS,n,2.0NRT,278.88,1,N


In [7]:
import geopandas
gdf = geopandas.GeoDataFrame(
    df_spain, geometry=geopandas.points_from_xy(df_spain.longitude, df_spain.latitude), crs="EPSG:4326"
)

# show top 3 records
gdf.head(3)

,country_id,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,geometry
0,ESP,37.35946,-5.86398,298.04,0.43,0.38,2024-12-01,228,N21,VIIRS,n,2.0NRT,282.51,1,N,POINT (-5.86398 37.35946)
1,ESP,37.40304,-5.75517,297.74,0.43,0.38,2024-12-01,228,N21,VIIRS,n,2.0NRT,278.88,1,N,POINT (-5.75517 37.40304)


Una vez podemos obtener dataframes de los paises que queramos, podemos mostrarlos en un mapa interactivo con folium.

In [11]:
import folium
import geopandas as gpd
import pandas as pd

shapefile_path = 'ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp'

MAP_KEY = 'a0f1afd59cc42db75820a429a7617f3a'
model = "VIIRS_NOAA21_NRT"
zone = "ESP"
days_num = 1
country_name = 'Spain'

area_url = f'https://firms.modaps.eosdis.nasa.gov/api/country/csv/{MAP_KEY}/{model}/{zone}/{days_num}'

# Descargar los datos desde la API
try:
    df_spain = pd.read_csv(area_url)
except Exception as e:
    print(f"Error al conectar con la API: {e}")
    df_spain = None

# Verificar si se descargaron los datos
if df_spain is not None and not df_spain.empty:
    # Convertir el DataFrame en un GeoDataFrame
    gdf = gpd.GeoDataFrame(
        df_spain,
        geometry=gpd.points_from_xy(df_spain.longitude, df_spain.latitude),
        crs="EPSG:4326"
    )

    # Cargar el shapefile de países
    world = gpd.read_file(shapefile_path)
    country = world[world['ADMIN'] == country_name]

    if country.empty:
        raise ValueError(f"No se encontró el país '{country_name}' en el shapefile.")

    # Reproyectar el país a un CRS proyectado para calcular el centroide correctamente
    country_projected = country.to_crs(epsg=3857)
    centroid = country_projected.geometry.centroid.to_crs(epsg=4326)

    # Calcular el centro del país para centrar el mapa
    map_center = centroid.iloc[0].y, centroid.iloc[0].x

    # Crear un mapa con Folium centrado en España
    m = folium.Map(location=map_center, zoom_start=6)

    # Añadir los límites de España al mapa
    folium.GeoJson(country, name="España").add_to(m)

    # Añadir puntos de incendios al mapa
    for _, row in gdf.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            color='red',
            fill=True,
            fill_opacity=0.6,
            tooltip=f"Fecha: {row['acq_date']}<br>Temperatura: {row['bright_ti4']}K"
        ).add_to(m)

    # Guardar el mapa
    m.save("mapa_incendios_espana.html")
    print("El mapa se ha guardado como 'mapa_incendios.html'. Ábrelo en tu navegador.")
else:
    print("No se obtuvieron datos de la API.")


El mapa se ha guardado como 'mapa_incendios.html'. Ábrelo en tu navegador.


In [10]:
import folium
import geopandas as gpd
import pandas as pd
from IPython.display import display, HTML  # Importar para mostrar mapas en el notebook

shapefile_path = 'ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp'

MAP_KEY = 'a0f1afd59cc42db75820a429a7617f3a'
model = "VIIRS_NOAA21_NRT"
zone = "ESP"
days_num = 1
country_name = 'Spain'

area_url = f'https://firms.modaps.eosdis.nasa.gov/api/country/csv/{MAP_KEY}/{model}/{zone}/{days_num}'

# Descargar los datos desde la API
try:
    df_spain = pd.read_csv(area_url)
except Exception as e:
    print(f"Error al conectar con la API: {e}")
    df_spain = None

# Verificar si se descargaron los datos
if df_spain is not None and not df_spain.empty:
    # Convertir el DataFrame en un GeoDataFrame
    gdf = gpd.GeoDataFrame(
        df_spain,
        geometry=gpd.points_from_xy(df_spain.longitude, df_spain.latitude),
        crs="EPSG:4326"
    )

    # Cargar el shapefile de países
    world = gpd.read_file(shapefile_path)
    country = world[world['ADMIN'] == country_name]

    if country.empty:
        raise ValueError(f"No se encontró el país '{country_name}' en el shapefile.")

    # Reproyectar el país a un CRS proyectado para calcular el centroide correctamente
    country_projected = country.to_crs(epsg=3857)
    centroid = country_projected.geometry.centroid.to_crs(epsg=4326)

    # Calcular el centro del país para centrar el mapa
    map_center = centroid.iloc[0].y, centroid.iloc[0].x

    # Crear un mapa con Folium centrado en España
    m = folium.Map(location=map_center, zoom_start=6)

    # Añadir los límites de España al mapa
    folium.GeoJson(country, name="España").add_to(m)

    # Añadir puntos de incendios al mapa
    for _, row in gdf.iterrows():
        intensity = row['bright_ti4']
        color = 'red' if intensity > 320 else 'orange' if intensity > 300 else 'yellow'
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            color=color,
            fill=True,
            fill_opacity=0.6,
            tooltip=f"Fecha: {row['acq_date']}<br>Temperatura: {intensity}K"
        ).add_to(m)

    # Mostrar el mapa dentro del notebook
    display(HTML(m._repr_html_()))
else:
    print("No se obtuvieron datos de la API.")
